<a href="https://colab.research.google.com/github/incognito-hb-24/Arthena/blob/main/Arthena_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai ipywidgets pandas matplotlib numpy
from IPython.display import clear_output, display
import pandas as pd, numpy as np, matplotlib.pyplot as plt, ipywidgets as widgets
import google.generativeai as genai
from google.colab import userdata
from datetime import datetime
clear_output()
print("Libraries ready.")

Libraries ready.


In [2]:
API_KEY = userdata.get("ARTHENA")
if not API_KEY:
    raise ValueError("No API key found! In Colab: Tools ▸ Secrets ▸ add key 'ARTHENA'.")

genai.configure(api_key=API_KEY)
print("Gemini API configured successfully!")

Gemini API configured successfully!


In [3]:
COLS = ["Date","Module","Type","Category","Entity","Amount (₹)","Notes"]
history_df = pd.DataFrame(columns=COLS)

def today():
    return datetime.today().strftime("%Y-%m-%d")

def summarize_context(df: pd.DataFrame) -> str:
    if df.empty:
        return "No transactions yet."
    inc = df[df["Type"]=="Income"]["Amount (₹)"].sum()
    exp = abs(df[df["Type"]=="Expense"]["Amount (₹)"]).sum()
    inv = abs(df[df["Type"]=="Investment"]["Amount (₹)"]).sum()
    net = inc - exp - inv
    return f"Income ₹{inc:.0f}, Expenses ₹{exp:.0f}, Investments ₹{inv:.0f}, Net Savings ₹{net:.0f}"


In [4]:
modules = ["Wealth","Cashflow"]
wealth_types = ["Asset","Liability","Goal","Insurance"]
cashflow_types = ["Income","Expense","Investment","Payment"]

mod_dd = widgets.Dropdown(options=modules, description="Module:")
type_dd = widgets.Dropdown(options=wealth_types, description="Type:")
cat_in = widgets.Text(description="Category:")
entity_in = widgets.Text(description="Entity:")
amt_in = widgets.FloatText(description="Amount (₹):")
date_in = widgets.DatePicker(description="Date:", value=datetime.today())
notes_in = widgets.Text(description="Notes:")
add_btn = widgets.Button(description="Add Entry", button_style="success")
out_add = widgets.Output()

def update_type(_):
    type_dd.options = wealth_types if mod_dd.value=="Wealth" else cashflow_types
mod_dd.observe(update_type, names="value")

def add_entry(_):
    global history_df
    with out_add:
        clear_output()
        if not entity_in.value or (amt_in.value is None) or amt_in.value == 0:
            print("Please fill Entity and a non-zero Amount.")
            return
        row = {
            "Date": date_in.value.strftime("%Y-%m-%d") if date_in.value else today(),
            "Module": mod_dd.value,
            "Type": type_dd.value,
            "Category": cat_in.value,
            "Entity": entity_in.value,
            "Amount (₹)": float(amt_in.value),
            "Notes": notes_in.value,
        }
        history_df = pd.concat([history_df, pd.DataFrame([row])], ignore_index=True)
        print("Entry added.")
        display(history_df.tail(5))

add_btn.on_click(add_entry)

display(widgets.VBox([
    widgets.HTML("<h3>Add Transaction</h3>"),
    mod_dd, type_dd, cat_in, entity_in, amt_in, date_in, notes_in,
    add_btn, out_add
]))


In [5]:
out_insight = widgets.Output()
refresh_btn = widgets.Button(description="Refresh Insights", button_style="info")

def render_insights(_=None):
    with out_insight:
        clear_output()
        if history_df.empty:
            print("No data yet.")
            return
        df = history_df.copy()
        inc = df[df["Type"]=="Income"]["Amount (₹)"].sum()
        exp = abs(df[df["Type"]=="Expense"]["Amount (₹)"]).sum()
        inv = abs(df[df["Type"]=="Investment"]["Amount (₹)"]).sum()
        net = inc - exp - inv
        print(f"Income: ₹{inc:,.0f} | Expenses: ₹{exp:,.0f} | Investments: ₹{inv:,.0f}")
        print(f"Net Savings: ₹{net:,.0f}")

        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        plt.figure(figsize=(6,3))
        vc = df["Type"].value_counts()
        plt.bar(vc.index, vc.values)
        plt.title("Transaction Types Distribution")
        plt.grid(axis="y", alpha=0.3)
        plt.show()

refresh_btn.on_click(render_insights)
display(widgets.VBox([
    widgets.HTML("<h3>Insights Dashboard</h3>"),
    refresh_btn, out_insight
]))


In [7]:
# Prefer these (no "models/" prefix)
PREFERRED_MODELS = [
    "gemini-1.5-flash",
    "gemini-1.5-flash-latest",
    "gemini-1.5-pro",
    "gemini-1.5-pro-latest",
]

# Try to construct a working model object
model_obj = None
chosen_name = None
for name in PREFERRED_MODELS:
    try:
        model_obj = genai.GenerativeModel(name)
        chosen_name = name
        break
    except Exception:
        continue

if not model_obj:
    raise RuntimeError(
        "Could not initialize a Gemini model. "
        "Check your API key/permissions or try a different model available to your account."
    )

print("Using Gemini model:", chosen_name)

chat_in = widgets.Textarea(
    placeholder="Ask Arthena anything about your finances…",
    layout=widgets.Layout(width='700px', height='80px')
)
chat_btn = widgets.Button(description="Ask Arthena", button_style="primary", layout=widgets.Layout(width='200px'))
chat_out = widgets.Output()
memory = []  # last few Q&As

def ask_arthena(_):
    with chat_out:
        clear_output()
        q = (chat_in.value or "").strip()
        if not q:
            print("Please type a question.")
            return
        context = summarize_context(history_df)
        recent = "\n".join([f"User: {u}\nArthena: {a}" for u, a in memory[-3:]])
        prompt = (
            "You are Arthena — a personal finance assistant.\n"
            "Use the user's data context to give simple, practical, India-friendly advice.\n"
            "Be concise (2–5 sentences), no generic filler.\n\n"
            f"Recent chat:\n{recent}\n\n"
            f"Data context: {context}\n\n"
            f"User question: {q}"
        )
        try:
            resp = model_obj.generate_content(prompt)
            ans = getattr(resp, "text", str(resp))
        except Exception as e:
            ans = f"Error: {e}"
        print("Arthena:\n", ans)
        memory.append((q, ans))

chat_btn.on_click(ask_arthena)
display(widgets.VBox([
    widgets.HTML("<h3>Arthena AI Chatbot</h3>"),
    chat_in, chat_btn, chat_out
]))


Using Gemini model: gemini-1.5-flash


In [8]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [9]:
from google.colab import output
output.disable_custom_widget_manager()

In [10]:
render_insights()
print("Arthena (Colab) is ready.")


Arthena (Colab) is ready.
